In [ ]:
import matplotlib.pyplot as plt    # for plotting the images
%matplotlib inline
import pandas as pd
import numpy as np    # for mathematical operations
import tensorflow as tf
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint,EarlyStopping
import os
from tensorflow.keras import Input, models, layers, optimizers, metrics
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator

np.random.seed(3)
tf.random.set_seed(3)

train_datagen=ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory('.', classes=[ 'train' ], target_size=(224, 224), batch_size=10, class_mode='categorical')

test_datagen=ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory('.', classes=[ 'test'], target_size=(224, 224), batch_size=10, class_mode='categorical')

transfer_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
transfer_model.trainable = False
transfer_model.summary()

model = Sequential()
model.add(transfer_model)
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

MODEL_DIR = './model/'
if not os.path.exists(MODEL_DIR):
    os.mkdir(MODEL_DIR)

modelpath="./model/{epoch:02d}-{val_loss:.4f}.hdf5"
# 모델 업데이트 및 저장
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=0, save_best_only=True)

# 학습 자동 중단 설정
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10)

history = model.fit(train_generator, epochs=100, validation_data=test_generator)

scores = model.evaluate(test_generator)
print("Real Test %s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
predictions = model.predict_classes(test_generator)
print("The screen time of NONE is", predictions[predictions==0].shape[0], "seconds")
print("The screen time of JERRY is", predictions[predictions==1].shape[0], "seconds")
print("The screen time of TOM is", predictions[predictions==2].shape[0], "seconds")

y_vloss = history.history['val_loss']
y_loss = history.history['loss']
acc = history.history['accuracy']
val_acc= history.history['val_accuracy']

x_len = np.arange(len(y_loss))
plt.plot(x_len, y_vloss, marker='.', c="red", label='Testset_loss')
plt.plot(x_len, y_loss, marker='.', c="blue", label='Trainset_loss')
plt.plot(x_len, val_acc, marker='.', c="pink", label='Testset_acc')
plt.plot(x_len, acc, marker='.', c="cornflowerblue", label='Trainset_acc')

plt.legend(loc='upper right')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss/acc')
plt.show()
